In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import panel as pn
import hvplot.xarray
import hvplot.pandas
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
import mysql.connector

pn.extension('tabulator')
pn.extension()
from sqlalchemy import create_engine

# Datensätze aus der MySQL (MariaDB) Datenbank auslesen

## Hilfsfunktion

In [53]:
def get_dataframe(table_name):
    con = mysql.connector.connect(host="localhost", database='moodle', user="root", passwd="",use_pure=True)

    cursor = con.cursor()
    query = "SELECT * FROM " + table_name
    cursor.execute(query)

    df = pd.DataFrame(cursor.fetchall())
    df.columns = cursor.column_names

    cursor.close()
    con.close()
    
    return df

## Datensätze aus der Datenbank

In [54]:
user = get_dataframe('mdl_user')
quiz = get_dataframe('mdl_quiz')
quiz_attempts = get_dataframe('mdl_quiz_attempts')
quiz_slots = get_dataframe('mdl_quiz_slots')
quiz_grades = get_dataframe('mdl_quiz_grades')
quiz_feedback = get_dataframe('mdl_quiz_feedback')
question = get_dataframe('mdl_question')
question_attempts = get_dataframe('mdl_question_attempts')
question_usages = get_dataframe('mdl_question_usages')
course = get_dataframe('mdl_course')
course_modules = get_dataframe('mdl_course_modules')
course_completion = get_dataframe('mdl_course_completions')
user_enrolments = get_dataframe('mdl_user_enrolments')
enrol = get_dataframe('mdl_enrol')
course_sections = get_dataframe('mdl_course_sections')

# Eigenen Datensatz für das Dashboard aufbauen

In [55]:
user_data = {'Name': user['firstname'] + ' ' + user['lastname'],
                      'SuS ID': user['id']
                     }

quiz_data = {'Aufgabe': quiz['name'],
             'Quiz ID': quiz['id'],
             'Maximale Punktzahl': quiz['sumgrades'],
             'Maximale Note in %': quiz['grade']
            }

quiz_attempts_data = {'SuS ID': quiz_attempts['userid'],
                      'Quiz ID': quiz_attempts['quiz'],
                      'Fragennutzung ID': quiz_attempts['uniqueid'],
                      'Quiz Startzeit': quiz_attempts['timestart'],
                      'Quiz Abgabezeit': quiz_attempts['timefinish'],
                      'Punktzahl': quiz_attempts['sumgrades'],
                      'Anzahl Versuche': quiz_attempts['attempt']
                     }

quiz_grade_data = {'Beste Punktzahl in %': quiz_grades['grade'],
                   'SuS ID': quiz_grades['userid'],
                   'Quiz ID': quiz_grades['quiz']
                  }

user_enrolments_data = {'Anmelde ID': user_enrolments['enrolid'],
                        'SuS ID': user_enrolments['userid']
                       }

enrol_data = {'Anmelde ID': enrol['id'],
              'Klassen ID': enrol['courseid']
             }

course_data = {'Klasse': course['fullname'],
               'Klassen ID': course['id']
              }

course_sections_data = {'Klassen ID': course_sections['course'],
                        'Fach': course_sections['name']
                       }

question_attempts_data = {'Frage Versuch ID': question_attempts['id'],
                          'Fragennutzung ID': question_attempts['questionusageid'],
                          'Slot': question_attempts['slot'],
                          'Frage ID': question_attempts['questionid'],
                          'Frage Text': question_attempts['questionsummary'],
                          'Antwort': question_attempts['responsesummary']
                         }

question_data = {'Frage ID': question['id'],
                 'Frage Name': question['name']
                }

In [47]:
pd.set_option('display.max_rows', 1000)

df_user = pd.DataFrame(user_data)

df_quiz_attempts = pd.DataFrame(quiz_attempts_data)
df_quiz_attempts['Quiz Startzeit'] = pd.to_datetime(df_quiz_attempts['Quiz Startzeit'], unit = 's')
df_quiz_attempts['Quiz Abgabezeit'] = pd.to_datetime(df_quiz_attempts['Quiz Abgabezeit'], unit = 's')
df_quiz_attempts['Quiz Bearbeitungsdauer'] = df_quiz_attempts['Quiz Abgabezeit'] - df_quiz_attempts['Quiz Startzeit']

df_quiz_grade = pd.DataFrame(quiz_grade_data)

#df_course_modules = pd.DataFrame(course_modules_data)

df_course = pd.DataFrame(course_data)

df_user_enrolments = pd.DataFrame(user_enrolments_data)

df_enrol = pd.DataFrame(enrol_data)

df_course_sections = pd.DataFrame(course_sections_data)

df_quiz = pd.DataFrame(quiz_data)

df_finished_quiz = pd.merge(df_user, df_quiz_attempts, on = 'SuS ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_quiz_grade, on = ['SuS ID', 'Quiz ID'], how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_user_enrolments, on = 'SuS ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_enrol, on = 'Anmelde ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_course, on = 'Klassen ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_course_sections, on = 'Klassen ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_quiz, on = 'Quiz ID', how = 'left')
df_finished_quiz['Beste Punktzahl in %'] = df_finished_quiz['Beste Punktzahl in %']*10
df_finished_quiz['Maximale Note in %'] = df_finished_quiz['Maximale Note in %']*10
df_finished_quiz = df_finished_quiz.drop([0,1,2,3,4,5,6,7])

In [56]:
# convert duration to 'HH:MM:SS'
# quelle: https://stackoverflow.com/questions/71128122/pandas-how-to-convert-timedelta64ns-to-hhmmss

def sec_to_format(s):
    if(not pd.isna(s)):
        h,s = divmod(int(s),3600)
        m,s = divmod(s,60)
        return f'{h:02}:{m:02}:{s:02}'   
    return np.nan

df_finished_quiz['Quiz Bearbeitungsdauer'] = [sec_to_format(s) for s in df_finished_quiz['Quiz Bearbeitungsdauer'].dt.total_seconds()]

AttributeError: Can only use .dt accessor with datetimelike values

In [57]:
# get good data types
df_finished_quiz['Punktzahl'] = df_finished_quiz['Punktzahl'].apply(float)

# Wichtige Listen

In [58]:
subject_list = course_sections['name'].unique().tolist()
subject_list.remove(None)
class_list = df_finished_quiz['Klasse'].unique().tolist()
exercise_list = quiz['name'].unique().tolist()
name_list = df_finished_quiz['Name'].unique().tolist()
name_list.sort()
class_list.sort()

# add general overview over all exercises
default_exercise = ['Allgemein']
df_finished_quiz[default_exercise] = default_exercise

class_list_with_none = ['Keine Auswahl'] + class_list
name_list_with_none = ['Keine Auswahl'] + name_list

# Funktionalitäten: Buttons und Menüs

In [59]:
exercise_button = pn.widgets.RadioButtonGroup(
    name='exercise_button', options = default_exercise + exercise_list, button_type='success', orientation = 'vertical')

subject_button = pn.widgets.RadioButtonGroup(
    name='Fach auswählen', options = subject_list, button_type='success')

class_selection = pn.widgets.Select(name='Klasse auswählen', options = class_list_with_none)

student_selection = pn.widgets.Select(name='Schüler*in auswählen', options = name_list_with_none)

start_date_picker = pn.widgets.DatetimePicker(name='Startdatum auswählen', value=dt.datetime(2023, 1, 1, 0, 0))

end_date_picker = pn.widgets.DatetimePicker(name='Enddatum auswählen', value=dt.datetime(2023, 12, 31, 0, 0))

# Datensouveränität

## Hilfsfunktion zum Löschen von Daten

In [60]:
# student_id: student
# start_time: start of period in which data needs to be deleted
# end_time: end of period that needs to be deleted
# question: column in which data needs to be deleted (will be replaced with NaN)
# index_list: row index in which data needs to be deleted (will be replaced with NaN)

def delete_data(student_id, start_time, end_time, question, index_list):
    # no end of period
    if end_time == 0:
        for index in index_list:
            # if quiz was done after choosing to not send data, replace value with NaN
            if ((df_finished_quiz.at[index,'Quiz Abgabezeit'] >= start_time) or pd.isna(df_finished_quiz.at[index,'Quiz Abgabezeit'])):
                df_finished_quiz.at[index, question] = np.NaN
                
                # also delete marks in percent
                if question == 'Punktzahl':
                    df_finished_quiz.at[index, 'Beste Punktzahl in %'] = np.NaN
    
    # end of period of time exists thus only delete necessary data in this period of time
    else:
        for index in index_list:
            # if quiz was done after choosing to not send data, replace value with NaN
            if ((df_finished_quiz.at[index,'Quiz Abgabezeit'] >= start_time and df_finished_quiz.at[index,'Quiz Abgabezeit'] <= end_time) or pd.isna(df_finished_quiz.at[index,'Quiz Abgabezeit'])):
                df_finished_quiz.at[index, question] = np.NaN
                
                if question == 'Punktzahl':
                    df_finished_quiz.at[index, 'Beste Punktzahl in %'] = np.NaN

## Erstellung des benötigten Datensatzes und Herauslesen der zu löschenden Daten

In [61]:
df_answers = pd.DataFrame(question_attempts_data)
df_answers_names = pd.DataFrame(question_data)
df_temp = df_finished_quiz[df_finished_quiz['Aufgabe'] == 'Auswahl der übermittelten Daten']

df_temp = pd.merge(df_temp, df_answers, on = 'Fragennutzung ID', how = 'left')
df_temp = pd.merge(df_temp, df_answers_names, on = 'Frage ID', how = 'left')
student_ids = df_temp['SuS ID'].unique().tolist()


for student_id in student_ids:
    i = 0
    times = df_temp['Quiz Abgabezeit'].unique().tolist()
    question_list = df_temp['Frage Name'].unique().tolist()
    index_list = df_finished_quiz.index[df_finished_quiz['SuS ID'] == student_id].tolist()

    # multiple periods of time need to be checked
    if len(times) >= 1:
        t = 0
        while (t < len(times)-1):
            for time in times:
                for question in question_list:
                    if df_temp.at[i, 'Antwort'] == 'Falsch':
                        delete_data(student_id, time, times[t+1], question, index_list)
                        i += 1
                    t += 1
                
        for time in times:
            for question in question_list:
                if df_temp.at[i, 'Antwort'] == 'Falsch':
                    delete_data(student_id, time, 0, question, index_list)
                i += 1
                
    # only one period of time needs to be checked
    else:
        for time in times:
            for question in question_list:
                if df_temp.at[i, 'Antwort'] == 'Falsch':
                    delete_data(student_id, time, 0, question, index_list)
                i += 1

# Graphen

In [62]:
df_finished_quiz

,Name,SuS ID,Quiz ID,Fragennutzung ID,Quiz Startzeit,Quiz Abgabezeit,Punktzahl,Anzahl Versuche,Quiz Bearbeitungsdauer,Beste Punktzahl in %,Anmelde ID,Klassen ID,Klasse,Fach,Aufgabe,Maximale Punktzahl,Maximale Note in %,Allgemein
8,Elijah Hickman,93,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Organisation,NaN,NaN,NaN,Allgemein
9,Elijah Hickman,93,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Biologie,NaN,NaN,NaN,Allgemein
10,Elijah Hickman,93,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Chemie,NaN,NaN,NaN,Allgemein
11,Jakeem Lamb,94,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Organisation,NaN,NaN,NaN,Allgemein
12,Jakeem Lamb,94,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Biologie,NaN,NaN,NaN,Allgemein
13,Jakeem Lamb,94,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Chemie,NaN,NaN,NaN,Allgemein
14,Camilla Ray,95,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Organisation,NaN,NaN,NaN,Allgemein
15,Camilla Ray,95,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Biologie,NaN,NaN,NaN,Allgemein
16,Camilla Ray,95,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Chemie,NaN,NaN,NaN,Allgemein
17,Kelly Berg,96,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,10.0,5.0,Klasse 8.2,Organisation,NaN,NaN,NaN,Allgemein


## Aufgaben: Anzahl der Punktzahlen

In [12]:
df_finished_quiz['Anzahl'] = df_finished_quiz.groupby(['Punktzahl'])['Klasse'].transform('count')
count_int = df_finished_quiz.interactive()

count_pipeline = (
    count_int[
        (count_int['Fach'] == subject_button) &
        ((count_int['Quiz Abgabezeit'] >= start_date_picker) & (count_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((count_int['Aufgabe'] == exercise_button) | (count_int['Allgemein'] == exercise_button)) &
        ((count_int['Klasse'] == class_selection) | (count_int['Name'] == student_selection))
    ]
    .groupby(['Punktzahl'])['Anzahl'].count()
    .to_frame()
    .reset_index()
    .sort_values(by='Punktzahl')
    .reset_index(drop=True)
)

In [13]:
count_plot = count_pipeline.hvplot.scatter(x='Punktzahl', y='Anzahl', title='Anzahl der Punktzahl', xlabel = 'Punktzahl', ylabel = 'Anzahl Schüler*innen').output()

## Aufgaben: Durchschnitte der Punktzahlen

In [14]:
df_finished_quiz['Durchschnitt Punktzahl'] = df_finished_quiz.groupby(['Aufgabe'])['Punktzahl'].transform('mean')
df_finished_quiz['Durchschnitt Fehlversuche'] = df_finished_quiz.groupby(['Aufgabe'])['Anzahl Versuche'].transform('mean')
mean_int = df_finished_quiz.interactive()

mean_pipeline = (
    mean_int[
        (mean_int['Fach'] == subject_button) &
        ((mean_int['Quiz Startzeit'] >= start_date_picker) & (mean_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((mean_int['Aufgabe'] == exercise_button) | (mean_int['Allgemein'] == exercise_button)) &
        ((mean_int['Klasse'] == class_selection) | (mean_int['Name'] == student_selection))
    ]
    .groupby(['Aufgabe', 'Durchschnitt Fehlversuche'])['Durchschnitt Punktzahl'].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

mean_table = mean_pipeline.pipe(pn.widgets.Tabulator, pagination = 'remote', page_size = 10, theme = 'fast', hidden_columns = ['index']).output()

# Create Teacher Dashboard

In [15]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Mein Dashboard', 
    sidebar=[pn.pane.Markdown("# Auswahl"),  
             class_selection,
             student_selection,
             exercise_button, 
             pn.pane.Markdown("## Einstellungen")],
    main=[pn.Row(pn.Column(subject_button),
                 pn.Column(start_date_picker, height = 350), 
                 pn.Column(end_date_picker)),
         pn.Row(pn.Column(count_plot), 
                pn.Column(mean_table))
         ],
    accent = '#88d8b0'
)

template.show()

#template.servable();
#command: panel serve --port 53369 learning_analytics_dashboard.ipynb in terminal then click on link

Launching server at http://localhost:62618
